# Modelo

Objetivo criar um modelo para definir o número de leitos necessário para uma unidade dependedo de uma projeção de demanda de leitos


Definições:
- número de leitos
- Tempo médio de internacao (média e sd) = Dias
- Tempo médio de chegada de novo paciente (média e sd) = horas
- ? Risco de óbito por dia para antes da internação
- ? Risco de óbito por dia para internado

Análise
- número total de solicitações
- porcentagem de solicitalções atendidas
- tempo médio para admissão do paciente
- ? mortalidade pré atendimento
- Cheques da definições

# Ciclo do paciente

- solicitação de vaga
- internacao na uti
- saida da uti


In [37]:
# versão 3
import simpy
import random
import numpy as np


class icusim():
    def __init__(self, env) -> None:
        
        self.verbose = False
        
        self.LEITOS_TOTAL = 10
        self.MEDIA_TEMPO_INTERNACAO = 7 #DIAS
        self.DESVIO_PADRAO_TEMPO_INTERNACAO = 1 # DIAS
        self.MEDIA_TEMPO_CHAGADA_PACIENTE =  6 #HORAS
        self.DESVIO_PADRAO_TEMPO_CHAGADA_PACIENTE = 1 # HORAS
        
        
        self.env = env
        self.pacientes_solicitacao = 0
        self.pacientes_internados = 0
        self.pacientes_atendidos = 0
        
        self.leitos = simpy.Resource(env, capacity=self.LEITOS_TOTAL)
        
        self.tabela_pacientes = {}
        
    def paciente(self, nome, tempo_internacao):
         
        t0 = self.env.now
        if self.verbose:
            print(f'Paciente {nome} solicitando UTI em {t0}')
        self.pacientes_solicitacao += 1
        leito_ocupado = self.leitos.request()
        
        yield leito_ocupado
        
        t1 = self.env.now
        # criar rotina para determinar se o tempo de espera para o leito foi muito longo e se o paciente ainda está na fila
        espera = (t1 - t0)/24
        print(f'Paciente {nome} esperou {(t1-t0)/24} dias')
        if espera > 5:
            self.atualiza_tabela(nome, espera=espera)
            return
        
        if self.verbose:
            print(f'Paciente {nome} Entrada na UTI em {t1}' )
        self.pacientes_solicitacao -= 1
        self.pacientes_internados += 1
        self.atualiza_tabela(nome, t0, t1)
        yield self.env.timeout(tempo_internacao)
        
        t2 = self.env.now
        if self.verbose:
            print(f'Paciente {nome} Saída da UTI em {t2}' )
        self.leitos.release(leito_ocupado)
        
        self.pacientes_atendidos += 1
        self.pacientes_internados -= 1
        self.atualiza_tabela(nome, t0, t1, t2)

    def atualiza_tabela(self, nome, t0=None, t1=None, t2=None, espera=None):
        self.tabela_pacientes[nome] = [t0, t1, t2, espera]
        

    def cria_paciente(self):
        paciente_nome = 0
        tempo_internacao = self.tempo_internacao()*24 # converte para horas 
        while True:
            # nesta linha tem o tempo de chegada de um novo paciente
            yield env.timeout(self.tempo_chegada_paciente())
            self.pacientes_solicitacao += 1
            t = self.env.now
            self.atualiza_tabela(paciente_nome, t0=t)
            
            if self.verbose:
                print(f'Paciente {paciente_nome} criado em {t}')
            self.env.process(self.paciente(paciente_nome, tempo_internacao))
            paciente_nome += 1

    def tempo_internacao(self):
        # impede tempos negativos
        tempo = -1
        while tempo < 0:
            tempo = np.random.normal(self.MEDIA_TEMPO_INTERNACAO, self.DESVIO_PADRAO_TEMPO_INTERNACAO)
        return tempo

    def tempo_chegada_paciente(self):
        # impede tempos negativos
        resposta = -1
        while resposta < 0:
            resposta = np.random.normal(self.MEDIA_TEMPO_CHAGADA_PACIENTE, self.DESVIO_PADRAO_TEMPO_CHAGADA_PACIENTE)
        return resposta


# unidade temporal é HORAS
            
env = simpy.Environment()
hospital = icusim(env)
env.process(hospital.cria_paciente())
env.run(until=365*24)

print(f'Pacientes solicitados: {hospital.pacientes_solicitacao}')
print(f'Pacientes internados: {hospital.pacientes_internados}')
print(f'Pacientes atendidos: {hospital.pacientes_atendidos}')
print(f'Pacientes na tabela: {hospital.tabela_pacientes}')


Paciente 0 esperou 0.0 dias
Paciente 1 esperou 0.0 dias
Paciente 2 esperou 0.0 dias
Paciente 3 esperou 0.0 dias
Paciente 4 esperou 0.0 dias
Paciente 5 esperou 0.0 dias
Paciente 6 esperou 0.0 dias
Paciente 7 esperou 0.0 dias
Paciente 8 esperou 0.0 dias
Paciente 9 esperou 0.0 dias
Paciente 10 esperou 4.9299526762944295 dias
Paciente 11 esperou 4.944490380730216 dias
Paciente 12 esperou 4.92305506841664 dias
Paciente 13 esperou 4.934769843286286 dias
Paciente 14 esperou 4.987551819396304 dias
Paciente 15 esperou 5.070941474242152 dias
Paciente 16 esperou 5.039680650357401 dias
Paciente 17 esperou 5.092600438428904 dias
Paciente 18 esperou 5.103007522586517 dias
Paciente 19 esperou 5.095976863312905 dias
Paciente 20 esperou 10.013968169562153 dias
Paciente 21 esperou 10.023932053951706 dias
Paciente 22 esperou 10.060523135098638 dias
Paciente 23 esperou 10.159265878017917 dias
Paciente 24 esperou 10.231275018052683 dias
Pacientes solicitados: 2883
Pacientes internados: 0
Pacientes atendido